# Анализ расхождений в отчетных данных: Дистрибьютор vs SFA MARS
### Анализ данных подтверждает расхождение в данных. 
Рассматривается 1-й период 2023 года, сеть "Деликатесные секреты"
Технические детали
Сумма по отчету дистрибьютора: 217 135 руб. (с НДС)
Сумма по данным SFA MARS: 146 573 руб. (без НДС)
Расхождение с учетом НДС: 70 562 руб. (32.5% от общей суммы)

Было выявлено – "Деликатесные секреты" работают с SFA MARS, идентификация торговых точек происходит по разным кодам в разных системах учета.

- Коды магазинов (storeCode) в рабочей таблице компании полностью отличаются от кодов в системе SFA MARS
- Товары поставляются в "Деликатесные секреты", но учитываются в разных системах по разным идентификаторам
- Компания, предоставляющая товары для "Деликатесные секреты", не интегрирована напрямую с системой SFA MARS

>Рекомендации –
Установить соответствие между кодами магазинов в разных системах учета для корректного сопоставления данных
Создать таблицу соответствия идентификаторов между системой компании и SFA MARS
Внедрить единую систему кодирования или автоматизированное сопоставление кодов при передаче данных
Разработать регламент сверки данных с учетом разницы в методологии учета (с НДС/без НДС)


#### Заключение
```
Основной причиной расхождений является использование разных систем кодирования торговых точек, а также различие в методологии учета (включение НДС). Хотя "Деликатесные секреты" работают с SFA MARS, компания-поставщик использует собственную систему кодирования, не интегрированную с SFA MARS напрямую, что приводит к неполному или некорректному отражению транзакций между системами.
```


In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
distributor_data_1 = pd.read_excel('../ distributor_report.xlsx', sheet_name='Ticket to technical support', skiprows=12) 
distributor_data_2 = pd.read_excel('../ distributor_report.xlsx', sheet_name='Sales SFA MARS Р1_2023') 
distributor_data_3 = pd.read_excel('../ distributor_report.xlsx', sheet_name='Sales Networ SFA MARS Р1_2023') 

# Переименование столбцов
distributor_data_1.columns = [
    '',
    'Продукт',
    'Неделя1_Количество',
    'Неделя1_СтоимостьНДС',  # Сумма = 217135
    'Неделя2_Количество',
    'Неделя2_СтоимостьНДС',
    'Неделя3_Количество',
    'Неделя3_СтоимостьНДС',
    'Неделя4_Количество',
    'Неделя4_СтоимостьНДС'
]
distributor_data_1.drop('', axis=1, inplace=True)

In [3]:
def process_table(df, column_name='Продукт'):
    # Создаем новый столбец для названий сетей
    df['network'] = None
    
    # Текущая сеть
    current_network = None
    
    # Проходим по всем строкам таблицы
    for i in range(len(df)):
        value = df.loc[i, column_name]
        
        # Проверяем, начинается ли значение со слова "сеть"
        if isinstance(value, str) and value.lower().startswith('сеть'):
            current_network = value
        
        # Заполняем столбец network текущим значением сети
        df.loc[i, 'network'] = current_network
    
    # Перемещаем столбец 'network' в начало таблицы
    columns = df.columns.tolist()
    columns.remove('network')
    df = df[['network'] + columns]
    
    return df

# Применяем функцию к DataFrame
distributor_data_1=process_table(distributor_data_1)

In [4]:
def clean_data(df):
    # 1. Удаляем строки, где значения в первых двух столбцах одинаковые
    columns = df.columns.tolist()
    first_col, second_col = columns[0], columns[1]
    df = df[df[first_col] != df[second_col]]
    
    # 2. Изменяем значения в первом столбце, извлекая текст в кавычках
    def extract_quoted_text(text):
        if not isinstance(text, str):
            return text
        
        # Ищем текст в кавычках
        match = re.search(r'"([^"]*)"', text)
        if match:
            return match.group(1)  # Возвращаем текст внутри кавычек
        return text  # Если кавычек нет, оставляем исходный текст
    
    # Применяем функцию к первому столбцу
    df[first_col] = df[first_col].apply(extract_quoted_text)
    
    return df

distributor_data_1 = clean_data(process_table(distributor_data_1))#[clean_data(process_table(distributor_data_1))['network']=='Деликатесные секреты']

/var/folders/b6/s36fmbbs5f78sybxrl1_y1300000gn/T/ipykernel_28957/940933300.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[first_col] = df[first_col].apply(extract_quoted_text)


In [5]:
# Данные из SFA MARS
sfa_data = pd.DataFrame({
    'TimePeriod': ['P1_2023', 'P2_2023'],
    'Network': ['Деликатесные секреты', 'Деликатесные секреты'],
    'SalesValue': [146573, 74549],
    'TimeYear': [2023, 2023],
    'Position': [10363763, 10363763]
})

In [6]:
last_col_3 = distributor_data_3.columns[-1]
second_last_col_3 = distributor_data_3.columns[-2]
last_col_2 = distributor_data_2.columns[-1]
second_last_col_2 = distributor_data_2.columns[-2]

distributor_data_3['%НДС'] = ((distributor_data_3[last_col_3] - distributor_data_3[second_last_col_3]) / distributor_data_3[second_last_col_3]) * 100
distributor_data_3['%НДС'].unique()

distributor_data_2['%НДС'] = ((distributor_data_2[last_col_2] - distributor_data_2[second_last_col_2]) / distributor_data_2[second_last_col_2]) * 100

In [7]:
distributor_data_2[distributor_data_2['Network']=='Деликатесные секреты']#.iloc[:, 5].sum()
distributor_data_2['%НДС'].unique()

quantity_columns = [col for col in distributor_data_1.columns if col.endswith('_Количество')]
# Создаем новый столбец, который будет содержать сумму этих столбцов
distributor_data_1['Общее количество'] = distributor_data_1[quantity_columns].sum(axis=1)
quantity_columns = [col for col in distributor_data_1.columns if col.endswith('_СтоимостьНДС')]
# Создаем новый столбец, который будет содержать сумму этих столбцов
distributor_data_1['СтоимостьНДС'] = distributor_data_1[quantity_columns].sum(axis=1)


distributor_data_1 = distributor_data_1.drop(distributor_data_1.columns[2:10], axis=1)

distributor_data_1 = distributor_data_1[distributor_data_1['network']=='Деликатесные секреты']#.unique()

In [8]:
# Функция для извлечения значения в скобках
def extract_bracket_value(text):
    if pd.isna(text):
        return None
    
    # Ищем значение в скобках
    match = re.search(r'\(([^)]+)\)$', text)
    if match:
        return match.group(1)
    else:
        return None

# Функция для удаления значения в скобках
def remove_bracket_value(text):
    if pd.isna(text):
        return text
    
    # Удаляем значение в скобках вместе со скобками
    return re.sub(r'\s*\([^)]+\)$', '', text)

# Создаем новый столбец со значением из скобок
distributor_data_1['Store code'] = distributor_data_1['Продукт'].apply(extract_bracket_value)

# Обновляем исходный столбец, удаляя значение в скобках
distributor_data_1['Продукт'] = distributor_data_1['Продукт'].apply(remove_bracket_value)

In [9]:
distributor_data_2 = distributor_data_2[distributor_data_2.iloc[:, 3] == 'Деликатесные секреты']
no_vat = int(distributor_data_3.iloc[:, -3].sum()) # сумма деликатесов без ндс
with_vat = int(distributor_data_1.iloc[:, -1].sum()) # сумма деликатесов в нашей таблице – с НДС
((with_vat - no_vat) / no_vat) * 100

3.794876853564335e+62

In [10]:
distributor_data_1['Store code'] = distributor_data_1['Store code'].astype(str)
distributor_data_2['Store code'] = distributor_data_2['Store code'].astype(str)
distributor_data_3['Store code'] = distributor_data_3['Store code'].astype(str)

distributor_data_3['Store code'].unique()

array(['171423', '114473', '170538', '360569', '118525', '274952',
       '170610', '464335', '114667', '263758', '289374', '474962',
       '441996', '360079', '244440', '208841', '273940'], dtype=object)

In [19]:
distributor_data_1.iloc[:, -1] = pd.to_numeric(distributor_data_1.iloc[:, -1], errors='coerce')
distributor_data_2.iloc[:, 0] = pd.to_numeric(distributor_data_2.iloc[:, 0], errors='coerce')
distributor_data_3.iloc[:, 0] = pd.to_numeric(distributor_data_3.iloc[:, 0], errors='coerce')
list1 = list(distributor_data_1['Store code'].unique())
list2 = list(distributor_data_3['Store code'].unique())
list3 = list(distributor_data_3['Store code'].unique())
list(set(list1) - set(list2))

[485057,
 556226,
 485062,
 520735,
 485069,
 485198,
 516848,
 485243,
 484988,
 485246,
 557311]

In [13]:
distributor_data_3['Product'].unique()

array(['Orbit Прохладная мята 13.6г', 'Orbit Сочный Арбуз13.6г',
       'Orbit Классический 10.2г', 'Orbit Клубника Банан13.6г',
       'Orbit Сладкая Мята 20*30*13.6г', 'Orbit Winterfresh 13.6г',
       'Orbit Белоснежный освежающая мята 13.6г',
       'Orbit Белоснежный классический 13.6г',
       'Orbit Экзотические ягоды 13.6г', 'Juicy Fruit 13г',
       'Wrigley Spearmint 13г'], dtype=object)

In [14]:
distributor_data_1

,network,Продукт,Общее количество,СтоимостьНДС,Store code
69,Деликатесные секреты,Juicy Fruit 5 пластинок 13г,400.0,11192.0,556226
70,Деликатесные секреты,Orbit д/детей Классическ 10.2г,460.0,18128.0,485057
71,Деликатесные секреты,Orbit Белосн классич 13.6г,660.0,22261.0,485246
72,Деликатесные секреты,Orbit Белосн oсвеж мята 13.6г,660.0,22261.0,485243
73,Деликатесные секреты,Orbit Клубника-Банан 13.6г,660.0,22261.0,485062
74,Деликатесные секреты,Orbit Сладкая мята 13.6г,660.0,22261.0,485069
75,Деликатесные секреты,Orbit Сочный арбуз 13.6г,660.0,22261.0,484988
76,Деликатесные секреты,Orbit Ягодный микс 13.6г,660.0,22261.0,520735
77,Деликатесные секреты,Orbit Winterfresh 13.6г,660.0,22261.0,485198
78,Деликатесные секреты,Wrigley's Spearmint 13г,360.0,9727.0,557311


In [15]:
distributor_data_2

,Store code,Store name,Store address,Network,"Sales Volume, Items","Sales Value, Price list MARS",Sales Value With VAT,%НДС
121,171423,"ООО ""Гастрономические сокровища""","Санкт-Петербург г, Ветеранов пр-кт, 50, к 2",Деликатесные секреты,600,16640,19966,19.987981
31266,114473,"ООО ""Гастрономические сокровища""","Карелия, Петрозаводск г, Ключевая (Ключевая р-...",Деликатесные секреты,300,8320,9983,19.987981
33275,170538,"ООО ""Гастрономические сокровища""","Ленинградская обл, Волховский р-н, Волхов г, М...",Деликатесные секреты,320,8786,10542,19.986342
49012,360569,"ООО ""Гастрономические сокровища""","Свердловская обл, Полевской г, Володарского ул...",Деликатесные секреты,260,7403,8883,19.991895
49093,118525,"ООО ""Гастрономические сокровища""","Ярославская обл, Ярославль г, Красноборская ул...",Деликатесные секреты,260,7403,8883,19.991895
49094,274952,"ООО ""Гастрономические сокровища""","Пензенская обл, Пенза г, Строителей пр-кт, 48",Деликатесные секреты,260,7403,8883,19.991895
49095,170610,"ООО ""Гастрономические сокровища""","Московская обл, Наро-Фоминск г, Апрелевка г, П...",Деликатесные секреты,300,8320,9983,19.987981
49096,464335,"ООО ""Гастрономические сокровища""","Курская обл, Курск г, Радищева ул, 18",Деликатесные секреты,320,8786,10542,19.986342
49100,114667,"ООО ""Гастрономические сокровища""","Краснодарский край, Туапсинский р-н, Туапсе г,...",Деликатесные секреты,300,8320,9983,19.987981
49103,263758,"ООО ""Гастрономические сокровища""","Саратовская обл, Саратов г, Большая Затонская ...",Деликатесные секреты,300,8320,9983,19.987981


In [16]:
distributor_data_3

,Store code,Network,Product,"Sales Volume, Items","Sales Value, Price list MARS",Sales Value With VAT,%НДС
0,171423,Деликатесные секреты,Orbit Прохладная мята 13.6г,60,1686.6,2023.79976,19.992871
1,171423,Деликатесные секреты,Orbit Сочный Арбуз13.6г,60,1686.6,2023.79976,19.992871
2,171423,Деликатесные секреты,Orbit Классический 10.2г,40,1313.6,1576.40016,20.006102
3,171423,Деликатесные секреты,Orbit Клубника Банан13.6г,60,1686.6,2023.79976,19.992871
4,171423,Деликатесные секреты,Orbit Сладкая Мята 20*30*13.6г,60,1686.6,2023.79976,19.992871
...,...,...,...,...,...,...,...
172,273940,Деликатесные секреты,Orbit Белоснежный освежающая мята 13.6г,30,843.3,1011.89988,19.992871
173,273940,Деликатесные секреты,Orbit Белоснежный классический 13.6г,30,843.3,1011.89988,19.992871
174,273940,Деликатесные секреты,Orbit Экзотические ягоды 13.6г,30,843.3,1011.89988,19.992871
175,273940,Деликатесные секреты,Juicy Fruit 13г,20,466.4,559.60008,19.982864


In [17]:
distributor_data_1.iloc[:, 1].unique()

array(['Juicy Fruit 5 пластинок 13г', 'Orbit д/детей Классическ 10.2г',
       'Orbit Белосн классич 13.6г', 'Orbit Белосн oсвеж мята 13.6г',
       'Orbit Клубника-Банан 13.6г', 'Orbit Сладкая мята 13.6г',
       'Orbit Сочный арбуз 13.6г', 'Orbit Ягодный микс 13.6г',
       'Orbit Winterfresh 13.6г', "Wrigley's Spearmint 13г",
       'Orbit Прохладная мята 13.6г'], dtype=object)

In [18]:
distributor_data_3.iloc[:, 2].unique()

array(['Orbit Прохладная мята 13.6г', 'Orbit Сочный Арбуз13.6г',
       'Orbit Классический 10.2г', 'Orbit Клубника Банан13.6г',
       'Orbit Сладкая Мята 20*30*13.6г', 'Orbit Winterfresh 13.6г',
       'Orbit Белоснежный освежающая мята 13.6г',
       'Orbit Белоснежный классический 13.6г',
       'Orbit Экзотические ягоды 13.6г', 'Juicy Fruit 13г',
       'Wrigley Spearmint 13г'], dtype=object)